In [1]:
import sys
import os
from typing import List, Optional

from pydantic import BaseModel
# Add the project root directory to Python path
sys.path.append(os.path.abspath('..'))

os.environ['toast_environment'] = 'preproduction'

from client.voice_ordering_client import VoiceOrderingClient
from client.service_session_provider import ServiceSessionProvider
from validation_agent.order_example_builder.order_case_api import GuestPersona, TargetOrder, OrderItem, OrderModification
from validation_agent.order_example_builder.order_case_generator import TestCaseGenerator, MenuApiClient, generate_and_save_test_cases
from validation_agent.order_example_builder.persona_loader import load_guest_personas
# from validation_agent.order_example_builder.example_usage import MockMenuApiClient
from validation_agent.config import config  # This loads your .env file


test_guid = "3997923f-e407-4a7f-aa97-cd5c19cbefee"




In [2]:
"""Example usage of the test case generation system."""

print("🌱 Loading persona seeds...")
personas = load_guest_personas()
print(f"Loaded {len(personas)} personas:")
for persona in personas:
    print(f"  - {persona.full_name} ({persona.order_complexity})")


🌱 Loading persona seeds...
Loaded 3 personas:
  - Sarah Johnson (simple)
  - Miguel Rodriguez (complex)
  - Emily Chen (simple)


In [3]:
# Replace this with your real menu API client
menu_api = MenuApiClient()
items = menu_api.get_menu_items()

for item in items:
    print(item)
    mods = menu_api.get_item_modifications(item)
    print(mods)
    break

import json
for mod in mods:
    print(f"Item: {mod.name}")
    print(f"  Price: {mod.price}")
    print(f"  GUID: {mod.guid}")
    print(f"  Group GUID: {mod.groupGuid}")
    print(f"  Out of Stock: {mod.outOfStock}")
    print(f"  Modifiers:")
    for modifier in mod.modifiers:
        print(f"    {modifier.name} (Required: {modifier.isRequired})")
        print(f"      Min/Max: {modifier.minimumSelections}/{modifier.maximumSelections}")
        for option in modifier.options:
            print(f"        - {option.name}: ${option.price} ({option.guid})")
    print()

name='Cheese Pizza' menu_item_guid='09a6cd9e-76ac-4823-a8ae-a18456186b4f' menu_group_guid='3bc22540-3d6a-4d79-8597-7c1eaa83a8c9' has_options=True price=None selected_options=None
[MenuItemDetail(name='Cheese Pizza', price=None, guid='09a6cd9e-76ac-4823-a8ae-a18456186b4f', groupGuid='3bc22540-3d6a-4d79-8597-7c1eaa83a8c9', modifiers=[ModifierGroup(name='Size', options=[ModifierOption(name='Small', price=9.99, guid='4fa2612f-0275-44f4-b4b3-482818dce990'), ModifierOption(name='Medium', price=11.99, guid='bc0d9a28-9fc1-47b2-a79b-85bc0d2badc7'), ModifierOption(name='Large', price=14.99, guid='42313203-7986-4d27-a2a3-6ba2f5704e9e')], isRequired=True, guid='a3dee65a-03fb-4e40-aa3b-a4307f67c3b9', minimumSelections=1, maximumSelections=1), ModifierGroup(name='Extras', options=[ModifierOption(name='Pepperoni', price=1.99, guid='1f974c77-6789-423c-b8ad-388df3f670e8'), ModifierOption(name='Sausage', price=1.99, guid='babb4e1b-d14b-49cf-8150-e5901ecee2cb'), ModifierOption(name='Gluten Free', price=4

### Conversation testing

In [9]:
###TODO

### Client Examples

In [4]:
menu_items_with_options = [
    {
        "menuItemGuid": item.menu_item_guid,  # API expects camelCase
        "menuGroupGuid": item.menu_group_guid  # API expects camelCase # might need to be renamed to 
    }
    for item in items if item.has_options
]

In [5]:
# 1. Get menu item details
service_session_provider = ServiceSessionProvider(
    config["CLIENT_ID"],
    config["CLIENT_SECRET"]
)

voClient = VoiceOrderingClient(service_session_provider)

menu_items = [
    {
        "menuItemGuid": "09a6cd9e-76ac-4823-a8ae-a18456186b4f",  # Cheese Pizza
        "menuGroupGuid": "3bc22540-3d6a-4d79-8597-7c1eaa83a8c9"
    }
]
restaurant_guid = "3997923f-e407-4a7f-aa97-cd5c19cbefee"
menu_details = voClient.getMenuItemDetails(restaurant_guid, menu_items_with_options)


In [6]:
for detail in menu_details:
    print(detail)

{'name': 'Cheese Pizza', 'price': None, 'guid': '09a6cd9e-76ac-4823-a8ae-a18456186b4f', 'groupGuid': '3bc22540-3d6a-4d79-8597-7c1eaa83a8c9', 'modifiers': [{'name': 'Size', 'options': [{'name': 'Small', 'price': 9.99, 'guid': '4fa2612f-0275-44f4-b4b3-482818dce990', 'isDefault': False, 'outOfStock': None}, {'name': 'Medium', 'price': 11.99, 'guid': 'bc0d9a28-9fc1-47b2-a79b-85bc0d2badc7', 'isDefault': False, 'outOfStock': None}, {'name': 'Large', 'price': 14.99, 'guid': '42313203-7986-4d27-a2a3-6ba2f5704e9e', 'isDefault': False, 'outOfStock': None}], 'isRequired': True, 'guid': 'a3dee65a-03fb-4e40-aa3b-a4307f67c3b9', 'minimumSelections': 1, 'maximumSelections': 1}, {'name': 'Extras', 'options': [{'name': 'Pepperoni', 'price': 1.99, 'guid': '1f974c77-6789-423c-b8ad-388df3f670e8', 'isDefault': False, 'outOfStock': None}, {'name': 'Sausage', 'price': 1.99, 'guid': 'babb4e1b-d14b-49cf-8150-e5901ecee2cb', 'isDefault': False, 'outOfStock': None}, {'name': 'Gluten Free', 'price': 4.99, 'guid': '